In [1]:
import os
import wandb
from pprint import pprint
from getpass import getpass
from wandb.integration.openai import autolog

import torch
import transformers

# Set OpenAI API key 

To get key, click on [link](https://platform.openai.com/account/api-keys).

In [10]:
# Set LLAMA2 API key environment variable
if os.getenv("OPENAI_API_KEY") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["LLAMA2_API_KEY"] = getpass("Paste your LLAM2 key from your huggingface settings \n")
  # openai.api_key = os.getenv("LLAMA2_API_KEY", "")

assert os.getenv("LLAMA2_API_KEY", "").startswith("hf_"), "This doesn't look like a valid OpenAI API key"
print("LLAMA2 API key configured")

Please enter password in the VS Code prompt at the top of your VS Code window!
LLAMA2 API key configured


In [17]:
# from transformers import LlamaForCausalLM, LlamaTokenizer

# model_id = 'meta-llama/Llama-2-70b-chat-hf'
# tokenizer = LlamaTokenizer.from_pretrained(model_id, token=os.getenv("LLAMA2_API_KEY", ""))
# model = LlamaForCausalLM.from_pretrained(model_id, token=os.getenv("LLAMA2_API_KEY", ""))


# # import transformers 

# # model_id = 'meta-llama/Llama-2-70b-chat-hf'

# # # begin initializing HF items, need auth token for these
# # hf_auth = os.getenv("LLAMA2_API_KEY", "")
# # model_config = transformers.AutoConfig.from_pretrained(
# #     model_id,
# #     use_auth_token=hf_auth
# # )


# Start W&B logging

autolog - convenient function for logging OpenAI results to W&B

In [4]:
autolog({"project":"llmapps", "job_type": "introduction"})

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: d-oliver-cort (doc93). Use `wandb login --relogin` to force relogin


# Tokenisation

The Llama 2 7B models were trained using the Llama 2 7B tokenizer - used to tokenise text and decode tokenised text

In [23]:
# Define llama2 model to load
model_id = 'meta-llama/Llama-2-70b-chat-hf'

# Get the HF auth token
hf_auth = os.getenv("LLAMA2_API_KEY", "")

In [24]:
# Example of tokanisation (encode and decode)
# - here we use the tokeniser for the llama2 model
# - different models may need a different tokeniser
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
    )
enc = tokenizer.encode("Weights & Biases is awesome!")
print(enc)
print(tokenizer.decode(enc))

/Users/davidoc/Documents/personal/LLM-powered-apps/venv_llm/lib/python3.8/site-packages/transformers/models/auto/tokenization_auto.py:631: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


[1, 1334, 5861, 669, 3457, 2129, 338, 29663, 29991]
<s> Weights & Biases is awesome!


In [25]:
# Decode tokens one by one
# - most natural way of splitting words into tokens would be to use the spaces
# - however, to contain the size of the vocabulary, some words split into sub-words (units)
for token_id in enc:
    print(f"{token_id}\t{tokenizer.decode([token_id])}")

1	<s>
1334	We
5861	ights
669	&
3457	Bi
2129	ases
338	is
29663	awesome
29991	!


# Sampling

### Sampling with Temperature

Let's sample some text from the model by passing the **temperature** parameter. 

In [7]:
# # Create a function that passes temperature parameter to the OpenAI completion function (API)
# def generate_with_temperature(temp):
#   "Generate text with a given temperature, higher temperature means more randomness"
#   response = openai.Completion.create(
#     model="text-davinci-003",
#     prompt="Say something about Weights & Biases",
#     max_tokens=50,
#     temperature=temp,
#   )
#   return response.choices[0].text

In [26]:
# Set quantization configuration to load large model with less GPU memory
# - this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

NameError: name 'torch' is not defined

In [ ]:
# Create a function that passes temperature parameter to the llama2 completion function (API)
def generate_with_temperature(temp):
  "Generate text with a given temperature, higher temperature means more randomness"
  response = openai.Completion.create(
    model="text-davinci-003",
    prompt="Say something about Weights & Biases",
    max_tokens=50,
    temperature=temp,
  )
  return response.choices[0].text

In [ ]:
# Generate text with different temperature values
# - with temperatures > 1, the generated text becomes too gibberish (cause more low probability tokens get sampled)
for temp in [0, 0.5, 1, 1.5, 2]:
  pprint(f'TEMP: {temp}, GENERATION: {generate_with_temperature(temp)}')

### Top p sampling

Can also use the [`top_p` parameter](https://platform.openai.com/docs/api-reference/completions/create#completions/create-top_p) to control the diversity of the generated text. 
- This parameter controls the cumulative probability of the next token. 
- For example, if `top_p=0.9`, the model will pick the next token from the top 90% most likely tokens. 
- The higher the `top_p` the more likely the model will pick a token that it hasn't seen before. 
- Decreasing `top_p` results in higher probability text being generated.
- You should only use one of `temperature` or `top_p` at a given time.

In [9]:
def generate_with_topp(topp):
  "Generate text with a given top-p, higher top-p means more randomness"
  response = openai.Completion.create(
    model="text-davinci-003",
    prompt="Say something about Weights & Biases",
    max_tokens=50,
    top_p=topp,
    )
  return response.choices[0].text

In [ ]:
for topp in [0.01, 0.1, 0.5, 1]:
  pprint(f'TOP_P: {topp}, GENERATION: {generate_with_topp(topp)}')

# Chat API

- The Chat API (using `gpt-3.5-turbo`) looks a bit different to the one above (used with the `davinci-003` model).
  
- The `gpt-3.5-turbo` model is faster and cheaper than `davinci-003`
  
- Instead of a prompt, this API takes a list of messages.
  - Messages come with different roles (system or user) and corresponding prompt
  - `system-role` gives some control over the model's response (below we steer the model to adhere to a certain behaviour)


- See [OpenAI ref](https://platform.openai.com/docs/guides/gpt)

In [ ]:
MODEL = "gpt-3.5-turbo"
response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Say something about Weights & Biases"},
    ],
    temperature=0,
)

# The response is a JSON object with relevant information about the request.
response

In [ ]:
pprint(response.choices[0].message.content)

Finish wandb run (see [Promps docs](https://docs.wandb.ai/guides/prompts))

In [ ]:
wandb.finish()